In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import re
import nbconvert
from IPython.display import display, HTML, display_html

# import custom functions
import sys
sys.path.append('/Users/dat/Library/CloudStorage/OneDrive-foryouandyourcustomers/GitHub/AutomatedPackagingCategories_Showcase/ml_packaging_classification/src')
import utils

# set formatting
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# ignore warnings
import warnings
warnings.filterwarnings('ignore')


SEED = 42

# Clean raw data

In [2]:
df_raw = pd.read_csv('../../data/data_showcase.csv', sep='\t')

In [3]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90035 entries, 0 to 90034
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Product Area             90035 non-null  object 
 1   Core Segment             90035 non-null  object 
 2   Brand                    90035 non-null  object 
 3   Material Number          90035 non-null  int64  
 4   Material No Text         90035 non-null  object 
 5   Component                90035 non-null  object 
 6   Material Description     90035 non-null  object 
 7   Packaging Code           90035 non-null  object 
 8   Material Characteristic  90035 non-null  object 
 9   Material Weight          82545 non-null  float64
 10  Column 21                90035 non-null  float64
 11  Weight measure           82545 non-null  object 
 12  Packaging Category       89262 non-null  object 
 13  Manufactoring Location   90035 non-null  object 
 14  Column 43             

In [4]:
df_raw.describe()

,Material Number,Material Weight,Column 21,Column 43
count,9.003500e+04,8.254500e+04,90035.000000,8.567100e+04
mean,5.482603e+07,4.581777e+02,2.674080,4.941166e+04
std,2.614605e+07,2.995114e+04,15.922722,2.016637e+05
min,1.000308e+07,0.000000e+00,0.000000,-3.014549e+04
25%,3.219510e+07,8.100000e-01,0.000000,1.722010e+03
50%,5.478846e+07,5.000000e+00,0.000000,7.460691e+03
75%,7.753107e+07,2.570000e+01,0.000000,2.997832e+04
max,9.999846e+07,2.300000e+06,328.000000,1.075813e+07


In [5]:
df_cleaned = df_raw.rename(columns={
    'Product Area': 'product_area',
    'Core Segment': 'core_segment',
    'Brand': 'brand',
    'Material Number': 'material_number',
    'Material No Text': 'material_number_text',
    'Component': 'component',
    'Material Description': 'component_text',
    'Packaging Code': 'packaging_code',
    'Material Characteristic': 'characteristic_value',
    'Material Weight': 'material_weight',
    'Column 21': 'col_21',
    'Weight measure': 'weight_measure',
    'Packaging Category': 'packaging_category',
    'Manufactoring Location': 'manufactoring_location',
    'Column 43': 'col_43'
})

In [6]:
df_cleaned['packaging_category'].value_counts(dropna=False)

P04 Hanger/ Clip                  13543
P03 Tube                          11687
B02 Blister and Insert Card        8744
P06 – TightPack                    8296
C01 - Folding carton               8219
B01 - Blister and sealed blist     7912
Unassigned                         5793
C12 - Corrugated carton            3872
C09 - Paperboard pouch             3478
C03 Trap Folding Card              2188
P05 Plastic Pouch                  1904
P01 - Plastic bag with header      1850
P08 Plastic Cassette               1708
P10 - Shrink film and insert o     1499
P07 Plastic Box                    1491
U0 – Unpacked                      1407
C06 Skincard                       1143
C02 Trap Card                       804
NaN                                 773
C07 Cardb. Sleeve w - w/o Shr.      676
C10 - Carton cover (Lid box)        652
-                                   492
P02 Case                            485
C05 Tray Packer                     431
C08 - Cardboard hanger w/o bag      400


In [7]:
# clean rows

df_cleaned['packaging_category'].mask(
    df_cleaned['packaging_category'].isin(['-', np.nan]), 'Unassigned', inplace=True
)

df_cleaned['packaging_category'].mask(
    df_cleaned['packaging_category'].isin(['No Packaging']), 'Unpacked', inplace=True
)
 
# df_cleaned = df_cleaned.loc[
#     (df_cleaned.weight_measure == 'G')
# ]


In [8]:
#  sort and filter columns
df_sub = df_cleaned[[
    'material_number',
    'material_number_text',
    'brand',
    'product_area',
    'core_segment',
    'component',
    'component_text',
    'manufactoring_location',
    'characteristic_value',
    'material_weight', 
    # 'weight_measure', # ignore since most values are 'G', 'NaN' might be an issue
    'packaging_code',
    'packaging_category',
    # 'col_21'
    # 'col_43'
]]

In [9]:
df_sub.head()

,material_number,material_number_text,brand,product_area,core_segment,component,component_text,manufactoring_location,characteristic_value,material_weight,packaging_code,packaging_category
0,45733343,Counter Display,BOT,PA5,Metal Grinding,6035765C21,Corrugated carton,Distribution Center,CORRUGATED,85.00,PCode_664273,D1 - Countertop display
1,45733343,Counter Display,BOT,PA5,Metal Grinding,6035940565,Label SB,Distribution Center,WOOD FREE,0.54,PCode_560426,D1 - Countertop display
2,45733343,Counter Display,BOT,PA5,Metal Grinding,6035822768,Tight -Pack label RB - 1ER,Distribution Center,MCB/GT2,22.90,PCode_481002,D1 - Countertop display
3,45733343,Counter Display,BOT,PA5,Metal Grinding,6035822768,Tight -Pack label RB - 1ER,Distribution Center,MCB/GT2,22.90,PCode_481002,D1 - Countertop display
4,45733343,Counter Display,BOT,PA5,Metal Grinding,6035765P54,Corrugated carton,Distribution Center,CORRUGATED,85.00,PCode_664273,D1 - Countertop display


In [10]:
df_ml = df_sub[df_sub.packaging_category != 'Unassigned']
df_no_packaging_categories = df_sub[df_sub.packaging_category == 'Unassigned']

In [11]:
df_ml.shape

(82977, 12)

In [12]:
df_no_packaging_categories.shape

(7058, 12)

In [13]:
df_ml.to_csv('../../data/output/df_ml.csv', sep='\t', index=False, header=True)

In [14]:
df_no_packaging_categories.to_csv('../../data/output/df_inf.csv', sep='\t', index=False, header=True)